In [1]:
import os
import sys

TOP_DIR=os.path.realpath("../../..")
if TOP_DIR not in sys.path: sys.path.append(TOP_DIR)

In [2]:
import pandas as pd
import numpy as np
from lib.util.convert import standardise_columns, literal_converter
from lib.util.geo import normalise_leeds_wards, ward_name_to_code

In [3]:
data = pd.read_csv('../../../working/metrics/community/events.csv', parse_dates=['Start date'])
data = data.rename(columns=standardise_columns)

In [4]:
wards = data['ward_(from_venue)'].fillna('[]').pipe(literal_converter).apply(np.unique)

In [5]:
data['ward_name'] = wards.apply(lambda x: x[0] if len(x) > 0 else np.nan).pipe(normalise_leeds_wards)
data['ward_code'] = data.ward_name.pipe(ward_name_to_code)
data['audience'] = data.actual_audience_size_number_of_participants_in_person
cleaned_data = data.loc[:, ['event_name', 'project_name', 'start_date', 'ward_name', 'ward_code', 'audience']]

In [6]:
DATA_DIR='../../../data/metrics/community'
os.makedirs(DATA_DIR, exist_ok=True)
cleaned_data.to_csv(os.path.join(DATA_DIR, 'events.csv'), index=False)

Missing or duplicated wards

In [7]:
data.loc[wards.apply(len) != 1, ['project_name', 'event_name', 'start_date', 'ward_(from_venue)']]

,project_name,event_name,start_date,ward_(from_venue)
49,01 - The Awakening,Creation Station - Leeds City Council,2022-11-23,NaN
50,01 - The Awakening,Creation Station - Leeds City Council,2022-11-24,NaN
64,01 - The Awakening,Creation Station - Yorkshire Schools Dance Fes...,2022-11-19,NaN
66,01 - The Awakening,Mass Movement Rehearsals,2023-01-05,NaN
67,01 - The Awakening,Waking the Artist in Communities,2023-02-06,NaN
...,...,...,...,...
265,Waking the Artist,Spring Gardens Pack,2023-03-23,NaN
266,Waking the Artist,The Den Headingley Pack,2023-02-13,NaN
267,Waking the Artist,Turning Lives Around Pack,2023-02-22,NaN
269,Waking the Artist,Wetherby Town Hall Pack,2023-02-25,NaN


Missing audience

In [8]:
data.loc[data.audience.isna(), ['project_name', 'event_name', 'start_date', 'audience']]

,project_name,event_name,start_date,audience
0,01 - The Awakening,Ballot Facilitated Session - Carr Manor Commun...,2022-11-01,NaN
1,01 - The Awakening,Ballot Facilitated Session - Chapel Allerton P...,2022-11-07,NaN
2,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-09,NaN
3,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-10,NaN
4,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-18,NaN
...,...,...,...,...
229,My World My City My Neighbourhood,Lincoln Greeners - Festival Closing,2023-08-24,NaN
230,My World My City My Neighbourhood,Lincoln Greeners - Festival Opening,2023-05-21,NaN
231,My World My City My Neighbourhood,Stand and Be Counted Round Table,2023-04-24,NaN
232,My World My City My Neighbourhood,The Ginnel,2023-10-13,NaN
